In [46]:
#import important stuff
from gpaw import GPAW, PW, FermiDirac
from ase.io import read
from ase.spectrum.band_structure import BandStructurePlot
from ase.spectrum.band_structure import BandStructure
from ase.build import mx2
import numpy as np
import matplotlib.pyplot as plt

In [47]:
formula = 'MoS2'
sym_path='KM'
bs1=BandStructure.read('formula_'+formula+'_direction_'+sym_path+'.json')
sym_path='KG'
bs2=BandStructure.read('formula_'+formula+'_direction_'+sym_path+'.json')

In [32]:
bs2

BandStructure(path=BandPath(path='Kpt0G', cell=[3x3], special_points={AGHKKpt0LM}, kpts=[30x3]), energies=[1x30x39 values], reference=-3.2218974665440108)

In [33]:
bs1

BandStructure(path=BandPath(path='Kpt0Kpt1', cell=[3x3], special_points={AGHKKpt0Kpt1LM}, kpts=[30x3]), energies=[1x30x39 values], reference=-3.229762968666191)

In [48]:
Nkpt=np.shape(bs1.energies)[1]
kPath1=bs1.path.kpts
kDiff1=kPath1-kPath1[0]
kPath2=bs2.path.kpts
kDiff2=kPath2-kPath1[0]
DeltaK_mag1=np.zeros((Nkpt))
DeltaK_mag2=np.zeros((Nkpt))
test_K=np.zeros((Nkpt))
for i in range(Nkpt):
    DeltaK_mag1[i]=np.sqrt((kPath1[i,0]-kPath1[0,0])**2+(kPath1[i,1]-kPath1[0,1])**2+(kPath1[i,2]-kPath1[0,2])**2)
    DeltaK_mag2[i]=np.sqrt((kPath2[i,0]-kPath2[0,0])**2+(kPath2[i,1]-kPath2[0,1])**2+(kPath2[i,2]-kPath2[0,2])**2)
    test_K[i]=np.linalg.norm(kDiff1[i])
#print(DeltaK_mag)
#print(test_K)#regner rigtig:)

In [49]:
energybands1=bs1.energies[0]
valenceBand1=energybands1[:,12]
conductionBand1=energybands1[:,13]
energybands2=bs2.energies[0]
valenceBand2=energybands2[:,12]
conductionBand2=energybands2[:,13]

In [58]:
#choose a cut off
k_cut=0.15
bool_k1=DeltaK_mag1<k_cut
bool_k2=DeltaK_mag2<k_cut
DeltaK_cut1=DeltaK_mag1[bool_k1]
valenceBand_cut1=valenceBand1[bool_k1]
conductionBand_cut1=conductionBand1[bool_k1]
DeltaK_cut2=DeltaK_mag2[bool_k2]
valenceBand_cut2=valenceBand2[bool_k2]
conductionBand_cut2=conductionBand2[bool_k2]

In [59]:
kVec1=np.concatenate((np.sort(-DeltaK_cut1),DeltaK_cut1[1:]))
valVec1=np.concatenate((np.sort(valenceBand_cut1),valenceBand_cut1[1:]))
conVec1=np.concatenate((-np.sort(-conductionBand_cut1),conductionBand_cut1[1:]))
print(kVec1)
print(valVec1)
print(conVec1)
kVec2=np.concatenate((np.sort(-DeltaK_cut2),DeltaK_cut2[1:]))
valVec2=np.concatenate((np.sort(valenceBand_cut2),valenceBand_cut2[1:]))
conVec2=np.concatenate((-np.sort(-conductionBand_cut2),conductionBand_cut2[1:]))
print(kVec2)
print(valVec2)
print(conVec2)

[-0.14629795 -0.13817029 -0.13004263 -0.12191496 -0.1137873  -0.10565963
 -0.09753197 -0.08940431 -0.08127664 -0.07314898 -0.06502131 -0.05689365
 -0.04876598 -0.04063832 -0.03251066 -0.02438299 -0.01625533 -0.00812766
 -0.          0.00812766  0.01625533  0.02438299  0.03251066  0.04063832
  0.04876598  0.05689365  0.06502131  0.07314898  0.08127664  0.08940431
  0.09753197  0.10565963  0.1137873   0.12191496  0.13004263  0.13817029
  0.14629795]
[-4.42393209 -4.40597721 -4.38786951 -4.36921895 -4.34969913 -4.32905951
 -4.30715403 -4.28393397 -4.25946901 -4.23396363 -4.20775385 -4.18133495
 -4.15536034 -4.13063008 -4.1080909  -4.08878939 -4.07380998 -4.06419507
 -4.06083408 -4.06419507 -4.07380998 -4.08878939 -4.1080909  -4.13063008
 -4.15536034 -4.18133495 -4.20775385 -4.23396363 -4.25946901 -4.28393397
 -4.30715403 -4.32905951 -4.34969913 -4.36921895 -4.38786951 -4.40597721
 -4.42393209]
[-1.85656711 -1.87660429 -1.90083811 -1.92894506 -1.96050869 -1.99507274
 -2.032132   -2.0711180

In [60]:
#units
import ase.units
Hartree = ase.units.Hartree#Hartree energy
Bohr = ase.units.Bohr#Bohr radius
print(Hartree)
print(Bohr)

27.211386024367243
0.5291772105638411


In [61]:
#fit
from scipy.optimize import curve_fit
yValData1=valVec1
yConData1=conVec1
yValData2=valVec2
yConData2=conVec2
#fit function
atomUnit_to_eV=Hartree
Aang_to_atomUnit=1/Bohr
#apparently k is already in atomic units!!!
def val_band(k,A1,m_eff_h):
    return A1-atomUnit_to_eV*(k)**2/(2*m_eff_h)
def con_band(k,A1,m_eff_e):
    return A1+atomUnit_to_eV*(k)**2/(2*m_eff_e)
parametersVal1, _ = curve_fit(val_band, kVec1, yValData1)
parametersCon1, _ = curve_fit(con_band, kVec1, yConData1)
print(parametersVal1)
print(parametersCon1)
parametersVal2, _ = curve_fit(val_band, kVec2, yValData2)
parametersCon2, _ = curve_fit(con_band, kVec2, yConData2)
print(parametersVal2)
print(parametersCon2)

#fit_val=val_band(xData,parameters1[0],parameters1[1])
#fit_con=con_band(xData,parameters2[0],parameters2[1])
#print(fit_val)
#print(valVec)
#print(fit_con)

[-4.10779337  0.79096035]
[-2.33837745  0.51316637]
[-4.06867152  0.76099371]
[-2.38692153  0.67602257]


In [ ]:
plt.plot(DeltaK_mag,valenceBand)
plt.plot(DeltaK_mag,conductionBand)
plt.legend(['Valence band','Conduction band'])
plt.xlabel('Delta k')
plt.ylabel('E')
plt.show()

In [ ]:
plt.plot(DeltaK_cut,valenceBand_cut)
plt.plot(DeltaK_cut,conductionBand_cut)
plt.legend(['Valence band','Conduction band'])
plt.xlabel('Delta k')
plt.ylabel('E')
plt.show()

In [24]:
print(len(DeltaK_cut))
print(valVec[len(DeltaK_cut)-1])
#ser godt ud

2
-4.060834081466296


In [ ]:
plt.plot(kVec,valVec)
plt.plot(kVec,conVec)
plt.legend(['Valence band','Conduction band'])
plt.xlabel('Delta k')
plt.ylabel('E')
plt.show()

In [20]:
#symmetric plot
#N_cut=len(DeltaK_cut)
#N=2*N_cut-1
#kVec=np.zeros((N))
#valVec=np.zeros((N))
#conVec=np.zeros((N))
#count=-1
#for i in range(N):
#    if i<N_cut:
#        kVec[i]=-DeltaK_cut[-1-i]
#        valVec[i]=valenceBand_cut[-1-i]
#        conVec[i]=conductionBand_cut[-1-i]
#    else:
#        count+=1
#        kVec[i]=DeltaK_cut[count]
#        valVec[i]=valenceBand_cut[count]
#        conVec[i]=conductionBand_cut[count]
#print(DeltaK_cut)
#print(kVec)
#print(valVec)
#print(conVec)
#print(len(kVec))
#

[0.         0.00812766 0.01625533 0.02438299 0.03251066 0.04063832
 0.04876598 0.05689365 0.06502131 0.07314898 0.08127664 0.08940431]
[-0.08940431 -0.08127664 -0.07314898 -0.06502131 -0.05689365 -0.04876598
 -0.04063832 -0.03251066 -0.02438299 -0.01625533 -0.00812766 -0.
  0.          0.00812766  0.01625533  0.02438299  0.03251066  0.04063832
  0.04876598  0.05689365  0.06502131  0.07314898  0.08127664]
[-4.28393397 -4.25946901 -4.23396363 -4.20775385 -4.18133495 -4.15536034
 -4.13063008 -4.1080909  -4.08878939 -4.07380998 -4.06419507 -4.06083408
 -4.06083408 -4.06419507 -4.07380998 -4.08878939 -4.1080909  -4.13063008
 -4.15536034 -4.18133495 -4.20775385 -4.23396363 -4.25946901]
[-2.07111801 -2.11142253 -2.15236023 -2.19318185 -2.23304383 -2.27101348
 -2.30607401 -2.3371209  -2.36301336 -2.38262897 -2.39496019 -2.39920275
 -2.39920275 -2.39496019 -2.38262897 -2.36301336 -2.3371209  -2.30607401
 -2.27101348 -2.23304383 -2.19318185 -2.15236023 -2.11142253]
23
